In [11]:
import pandas as pd
import joblib
from sklearn.preprocessing import LabelEncoder

In [12]:
df=pd.read_csv(r'D:\Weaather Forecast\Dataset\daily_training_table.csv')			
df.head()

,date,temp_c,pressure,light_lux,rain_rate,humidity,wind_speed,cloud_info,month,dayofweek,temp_c_lag1,pressure_lag1,light_lux_lag1,rain_rate_lag1,humidity_lag1,wind_speed_lag1,target_cloud,target_y
0,2024-01-04,27.138309,1001.881221,7796.808824,0.687986,49.247500,0.047868,bright,1,3,25.384526,1010.497239,8104.968421,0.000000,60.474842,0.000000,dim,2
1,2024-01-05,29.333627,993.799930,13249.948187,0.804056,47.145440,0.093057,dim,1,4,27.138309,1001.881221,7796.808824,0.687986,49.247500,0.047868,dim,2
2,2024-01-06,26.443684,996.298658,12428.142857,0.000000,55.533759,0.002632,dim,1,5,29.333627,993.799930,13249.948187,0.804056,47.145440,0.093057,dim,2
3,2024-01-07,27.235630,998.820094,13279.992593,0.000000,56.539852,0.001778,dim,1,6,26.443684,996.298658,12428.142857,0.000000,55.533759,0.002632,bright,0
4,2024-01-08,26.250851,1001.936932,6690.542553,0.000000,49.797766,0.000000,bright,1,0,27.235630,998.820094,13279.992593,0.000000,56.539852,0.001778,dim,2


In [20]:
model = joblib.load("weather_model.pkl")

In [21]:
label_encoders = {}
for col in df.select_dtypes(include=["object"]).columns:
    if col != "date":
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])
        label_encoders[col] = le

In [22]:
feature_cols = [col for col in df.columns if col not in ["date", "target_y"]]

In [23]:
last_row = df.iloc[-1].copy()

In [24]:
predictions = []

In [25]:
for i in range(7):  # forecast 7 days
    # Only training features (16 cols)
    X_input = last_row[feature_cols].values.reshape(1, -1)

    # Predict target_y
    pred_y = model.predict(X_input)[0]

    # Create new row
    new_row = last_row.copy()
    new_row["target_y"] = pred_y

    # Advance date
    new_date = pd.to_datetime(last_row["date"]) + pd.Timedelta(days=1)
    new_row["date"] = new_date.strftime("%Y-%m-%d")

    # Store prediction
    predictions.append(new_row)

    # Update for next loop
    last_row = new_row

C:\Users\Jashwanth\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Jashwanth\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Jashwanth\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Jashwanth\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Jashwanth\AppData\Local\Programs\Python\Python312\Lib\s

In [26]:
forecast_df = pd.DataFrame(predictions)
forecast_df.to_csv("weekly_forecast.csv", index=False)

In [27]:
print("✅ Weekly forecast saved as weekly_forecast.csv")

✅ Weekly forecast saved as weekly_forecast.csv
